In [28]:
import numpy as np
import pandas as pd

file_path = r"C:\Users\ryan_\OneDrive\Desktop\Bipolar\Data\bipolar_dataset.csv"

# Define column names based on the first line of the file
with open(file_path, "r") as f:
    columns = f.readline().strip().split(";")

# Read the file into a DataFrame
df = pd.read_csv(file_path, sep=";", skiprows=1, header=None, encoding='latin1')

# Assign column names
df.columns = columns

# Display the DataFrame
df.head()
df.shape
df.columns
df.info

<bound method DataFrame.info of           subreddit              author        date  \
0     bipolarreddit     TanagerOffTrack  01/01/2018   
1     bipolarreddit    DuplicitousSnake  01/01/2018   
2     bipolarreddit         dearangelou  01/01/2018   
3     bipolarreddit        dootdoopdooq  01/01/2018   
4     bipolarreddit        lemonyellow3  01/01/2018   
...             ...                 ...         ...   
1687  bipolarreddit       LilyChickadee  20/04/2019   
1688  bipolarreddit  pineapplesforbfast  20/04/2019   
1689  bipolarreddit     FrannyCaulfield  20/04/2019   
1690  bipolarreddit           coconutfi  20/04/2019   
1691  bipolarreddit        gangstergaga  20/04/2019   

                                                   post  \
0     What birth control are you on? I tried an IUD ...   
1     Hypersexuality I was diagnosed with bipolar un...   
2     Meditation I have been using the free meditati...   
3     Family question diagnosis - vent Start with th...   
4     Unable